In [1]:
import pandas as pd
import numpy as np
import os
import re
pd.set_option('precision',2)

def day(l):
    a = str(l)
    return a[:10]

#Daily averages must first be calculated using Day-of-the-Week-Average.ipynb
#Copy averages into an appropriately formatted .xlsx file (see technical doc)
def daycountave(filepath1):
    df = pd.read_excel(filepath1)
    df2 = pd.read_excel('INSERT PATH FOR AVERAGE SPREADSHEET HERE')
    df = df[df['LANGUAGE']=='en'].copy()
    df['bow'] = df['TEXT'].str.lower().str.replace(r'(https?://.+|[^\w#@]+|\d+)+',' ').str.split()
    df = df[df['bow'].isnull()==False].copy()
    df = df[(df['SOURCE'].str.contains('bot', case=False)==False) | (df['SOURCE'].str.contains('tweetbot', case=False)==True)]
    df['Day'] = df['CREATED_AT_LOCAL'].apply(day)
    days = df.pivot_table(index='Day',values='TEXT',aggfunc="count")
    city = df['CITY'][0]
    def addave(l):
        return df2.loc[l][city]
    days.columns = ['Angry Tweets']
    days['Days'] = days.index
    days['Days'] = pd.to_datetime(days['Days'])
    days['Weekday'] = days['Days'].dt.weekday_name
    days['Weekday Ave'] = days['Weekday'].apply(addave)
    days['Difference'] = days['Angry Tweets']-days['Weekday Ave']
    days['% Difference'] = (days['Difference']/days['Weekday Ave'])*100
    start = df['CREATED_AT_LOCAL'].iloc[-1]
    end = df['CREATED_AT_LOCAL'].iloc[0]
    name = city + ' ' + str(start)[:10] + ' to ' + str(end)[:10] + ' vs ave' '.csv'
    days.to_csv(os.path.join(os.path.dirname(filepath1),name), columns = ['Weekday','Angry Tweets','Difference','% Difference'])
    
def processfolder(directory, function):
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx"):
            function(os.path.join(directory, filename))
            continue
        else:
            continue